In [1]:
import folium
import ee

In [2]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

In [3]:
ee.Initialize()
gfc2015 = ee.Image('UMD/hansen/global_forest_change_2015')
# make a map
emap = EngineMap()

# add the raster
emap.add_ee_raster(
    image=gfc2015,
    name='treecover2000',
    attr="Tian",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['black','white'],
    }
)

emap.add_ee_raster(
    image=gfc2015,
    name='false color',
    attr="Tian",
    vis_params={
        "bands": ['last_b50', 'last_b40', 'last_b30'],
    }
)

emap.add_ee_raster(
    image=gfc2015,
    name='green',
    attr="Tian",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain']
    }
)

emap.add_ee_raster(
    image=gfc2015,
    name='forest cover, loss, gain',
    attr="Tian",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain'],
        "max": [1, 255, 1]
    }
)

emap.add_ee_raster(
    image=gfc2015,
    name='forest cover palette',
    attr="Tian",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['000000', '00FF00']
    }
)

emap.add_ee_raster(
    image=gfc2015,
    name='forest cover percent',
    attr="Tian",
    vis_params={
        "bands": ['treecover2000'],
        "max": 100,
        "palette": ['000000', '00FF00']
    }
)

emap.add_child(folium.LayerControl())

In [4]:
# created separate map to show transparency
emap2 = EngineMap()
emap2.add_ee_raster(
    image=gfc2015.mask(gfc2015),
    name='forest cover masked',
    attr="Tian",
    vis_params={
        "bands": ['treecover2000'],
        "max": 100,
        "palette": ['000000', '00FF00']
    }
)

In [5]:
final_map = EngineMap()
tree_cover = gfc2015.select('treecover2000')
loss_image = gfc2015.select('loss')
gain_image = gfc2015.select('gain')
final_map.add_ee_raster(
    image=tree_cover.updateMask(tree_cover),
    name='Forest Cover',
    attr="Tian",
    vis_params={
        "max": 100,
        "palette": ['000000', '00FF00']
    }
)

final_map.add_ee_raster(
    image=loss_image.updateMask(loss_image),
    name='Loss',
    attr="Tian",
    vis_params={
        "palette": ['FF0000']
    }
)

final_map.add_ee_raster(
    image=gain_image.updateMask(gain_image),
    name='Gain',
    attr="Tian",
    vis_params={
        "palette": ['0000FF']
    }
)